In [1]:
import numpy as np
import xarray as xr
from tqdm import tqdm

In [2]:
path2pcip = '/home/swenson/era5_data_direct_1.0degree_Version3/tp/ERA5_1980-2010_tp_3H_percentiles.nc'
pcip_dat = xr.open_dataset(path2pcip)
WDper = pcip_dat['WD_percentile']
tp = pcip_dat['tp']

In [3]:
path = './'
cape_dat = xr.open_dataset(path+"CAPE_consumption_percentile.nc")
cape = cape_dat['CAPE_consumption_percentile'].sel(time=slice('1980-01-01', '2011-01-01'))
frt_dat  = xr.open_dataset(path+"gradEPT_Frontal_percentile.nc")
frt = frt_dat['gradEPT_percentile_AboveMedian'].sel(time=slice('1980-01-01', '2011-01-01'))
vrt_dat  = xr.open_dataset(path+"PositiveRelativeVorticityAdvection_Percentile.nc")
vrt = vrt_dat['PositiveRelativeVorticityAdvection_Percentile'].sel(time=slice('1980-01-01', '2011-01-01'))
Avrt_dat  = xr.open_dataset(path+"PositiveTotalVorticityAdvection_Percentile.nc")
Avrt = Avrt_dat['PositiveTotalVorticityAdvection_Percentile'].sel(time=slice('1980-01-01', '2011-01-01'))

TIME = cape['time'].dt
LAT = cape['latitude']
LON = cape['longitude']

In [4]:
nmap = np.load("/home/swenson/projects/AGU2020/SOM_Zones/ERA5_1deg_regions7_EdgesRemoved.npy")

In [5]:
PTHRESH = 95
for region in tqdm([1,2,3,4,5,6,7]): #[1,2,3,4,5,6,7]
    region_i = np.where(nmap.reshape(-1) == region)
    PERCENT = np.squeeze(WDper.values.reshape((100,-1),order='c')[:,region_i])
    PERCENT = PERCENT[PTHRESH,:]
    
    for season in ['DJF','MAM','JJA','SON']: #['DJF','MAM','JJA','SON']
        time_i = np.where(TIME.season == season)[0]
        
        pcip_temp = tp[time_i,:,:]
        
        pcip_temp = np.squeeze(pcip_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
        
        isPEX = np.where(pcip_temp > np.broadcast_to(PERCENT,pcip_temp.shape))
        
        cape_temp = cape[time_i,:,:]
        cape_temp = np.squeeze(cape_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
        frt_temp = frt[time_i,:,:]
        frt_temp = np.squeeze(frt_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
        vrt_temp = vrt[time_i,:,:]
        vrt_temp = np.squeeze(vrt_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
        avrt_temp = Avrt[time_i,:,:]
        avrt_temp = np.squeeze(avrt_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
        
        capeEX = cape_temp[isPEX[0],isPEX[1]]
        vrtEX = vrt_temp[isPEX[0],isPEX[1]]
        frtEX = frt_temp[isPEX[0],isPEX[1]]
        avrtEX = avrt_temp[isPEX[0],isPEX[1]]
        
        coords = dict(

            events = (['events'],np.arange(len(isPEX[0])))

        )
        
        data_vars=dict(

            pcip = (['events'], pcip_temp[isPEX[0],isPEX[1]]),
            cape = (['events'], capeEX),
            vrt  = (['events'], vrtEX),
            frt  = (['events'], frtEX),
            avrt = (['events'], avrtEX)

            )

        var_out = xr.Dataset(

            data_vars=data_vars,
            coords=coords

            )
        
        var_out.to_netcdf(path+'DistBySeason/MetricScores_CapeVrtFrt_Region'+str(region)+'_'+season+'.nc')
        
    for hour in [0,3,6,9,12,15,18,21]: #[0,3,6,9,12,15,18,21]
        time_i = np.where(TIME.hour == hour)[0]
        
        pcip_temp = tp[time_i,:,:]
        
        pcip_temp = np.squeeze(pcip_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
        
        isPEX = np.where(pcip_temp > np.broadcast_to(PERCENT,pcip_temp.shape))
        
        cape_temp = cape[time_i,:,:]
        cape_temp = np.squeeze(cape_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
        frt_temp = frt[time_i,:,:]
        frt_temp = np.squeeze(frt_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
        vrt_temp = vrt[time_i,:,:]
        vrt_temp = np.squeeze(vrt_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
        avrt_temp = Avrt[time_i,:,:]
        avrt_temp = np.squeeze(avrt_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
        
        capeEX = cape_temp[isPEX[0],isPEX[1]]
        vrtEX = vrt_temp[isPEX[0],isPEX[1]]
        frtEX = frt_temp[isPEX[0],isPEX[1]]
        avrtEX = avrt_temp[isPEX[0],isPEX[1]]
        
        coords = dict(

            events = (['events'],np.arange(len(isPEX[0])))

        )
        
        data_vars=dict(

            pcip = (['events'], pcip_temp[isPEX[0],isPEX[1]]),
            cape = (['events'], capeEX),
            vrt  = (['events'], vrtEX),
            frt  = (['events'], frtEX),
            avrt = (['events'], avrtEX)

            )

        var_out = xr.Dataset(

            data_vars=data_vars,
            coords=coords

            )
        
        var_out.to_netcdf(path+'DistByTime/MetricScores_CapeVrtFrt_Region'+str(region)+'_Hour'+str(hour)+'.nc')
        
    for season in ['DJF','MAM','JJA','SON']: #['DJF','MAM','JJA','SON']
        for hour in [0,3,6,9,12,15,18,21]: #[0,3,6,9,12,15,18,21]
            
            time_i = np.where(np.logical_and(TIME.hour == hour, TIME.season == season))[0]
        
            pcip_temp = tp[time_i,:,:]

            pcip_temp = np.squeeze(pcip_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])

            isPEX = np.where(pcip_temp > np.broadcast_to(PERCENT,pcip_temp.shape))

            cape_temp = cape[time_i,:,:]
            cape_temp = np.squeeze(cape_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
            frt_temp = frt[time_i,:,:]
            frt_temp = np.squeeze(frt_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
            vrt_temp = vrt[time_i,:,:]
            vrt_temp = np.squeeze(vrt_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])
            avrt_temp = Avrt[time_i,:,:]
            avrt_temp = np.squeeze(avrt_temp.values.reshape((len(time_i),-1),order='c')[:,region_i])

            capeEX = cape_temp[isPEX[0],isPEX[1]]
            vrtEX = vrt_temp[isPEX[0],isPEX[1]]
            frtEX = frt_temp[isPEX[0],isPEX[1]]
            avrtEX = avrt_temp[isPEX[0],isPEX[1]]
            
            coords = dict(

            events = (['events'],np.arange(len(isPEX[0])))

            )

            data_vars=dict(

                pcip = (['events'], pcip_temp[isPEX[0],isPEX[1]]),
                cape = (['events'], capeEX),
                vrt  = (['events'], vrtEX),
                frt  = (['events'], frtEX),
                avrt = (['events'], avrtEX)

                )

            var_out = xr.Dataset(

                data_vars=data_vars,
                coords=coords

                )

            var_out.to_netcdf(path+'DistByTimeAndSeason/MetricScores_CapeVrtFrt_Region'+str(region)+'_'+season+'_Hour'+str(hour)+'.nc')
        
        
        
        
        

100%|██████████| 7/7 [13:07<00:00, 112.51s/it]
